In [1]:
import bs4
import pandas as pd
import lxml
from icecream import ic
import string
import requests
import json
import os
from tqdm import tqdm
import pickle

In [6]:
df = pd.DataFrame(columns = ['ID', 'ФИО', 'Дата рождения', 'Дата смерти', 'Сан','Память', 'Биография', 
                             'Публикации', 'Архивные источники', 'Заявители'])

In [7]:
df

,ID,ФИО,Дата рождения,Дата смерти,Сан,Память,Биография,Публикации,Архивные источники,Заявители


In [2]:
for i in tqdm(range(106562, 112822)):
    url = f"https://pstgu.ru/baza-dannykh-za-khrista-postradavshie/ebs-za-khrista-postradavshie/a/{i}/"
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'pstgu_base{i}.html', 'w', encoding='utf-8') as f:
            f.write(response.text)

100%|████████████████████████████████████████████████████████████████████████████| 5437/5437 [1:39:48<00:00,  1.10s/it]


In [50]:
path = 'pages'
pstgu_base = []
info = {}
for i, file in tqdm(zip(range(106562, 112822), os.listdir(path=path))):
    with open(f'{path}/{file}', encoding='utf-8') as f:
        html = f.read()
    soup = bs4.BeautifulSoup(html, 'html.parser')
    fio = soup.find('h2', class_='inner-page__title').text
    for tag in soup.find_all('span'):
        if 'Дата рождения' in tag.text:
            birth_date = tag.text.split('Дата рождения')[1].strip()
        if 'Дата смерти' in tag.text:
            death_date = tag.text.split('Дата смерти')[1].strip()
        if 'Память' in tag.text:
            memory = tag.text.split('Память')[1].strip()
        if 'Сан' in tag.text:
            san = tag.text.split('Сан')[1]
    for tag_ in soup.find_all('div', class_="accordion__item"):
        if 'Биография' in tag_.contents[1].text:
            biography = re.sub('\s+', ' ', tag_.contents[3].text).strip()
        if 'Публикации' in tag_.contents[1].text:
            publications = re.sub('\s+', ' ', tag_.contents[3].text).strip()
        if 'Архивные источники' in tag_.contents[1].text:
            archive = re.sub('\s+', ' ', tag_.contents[3].text).strip()
        if 'Заявители' in tag_.contents[1].text:
            applicants = re.sub('\s+', ' ', tag_.contents[3].text).strip()
    info = {'ID': i, 'ФИО': fio, 'Дата рождения': birth_date, 'Дата смерти': death_date, 'Память': memory, 'Сан': san, 
                 'Биография': biography, 'Публикации': publications, 'Архивные источники': archive, 'Заявители': applicants}
    pstgu_base.append(info)

6260it [12:16,  8.50it/s]


In [54]:
for dict_ in pstgu_base:
    dict_['Сан'] = dict_['Сан'].strip(', ')

In [56]:
import json
with open('pstgu_base.json', 'w', encoding='utf-8') as f:
    json.dump(pstgu_base, f, ensure_ascii=False, indent=4)

In [ ]:
pstgu_base

In [10]:
unique_card_tags

{'Дата рождения', 'Дата смерти', 'Память', 'Сан'}